In [38]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
cleaned_data_1=pd.read_csv('cleaned_dataset_1.csv')
cleaned_data_1

,Unnamed: 0.1,Unnamed: 0,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Total_Youtube,Total_Netflix,Total_Gaming,Total_social_media,Total_Google,Total_Email,Total_Other,StartDate,StartTime,EndDate
0,0,0,9223372036854775807,770,662,1823652,208201448079117,33664962239,35521209507511,9.16456699548519E+015,...,18355943,17855187,292426453,1570185,2905912,40313283,180558843,2019-04-04,14:35:00,2019-04-25
1,1,1,9223372036854775807,235,606,1365104,208201909211140,33681854413,35794009006359,L77566A,...,39359124,35565545,609920783,1933278,4414096,54429437,541959383,2019-04-09,08:15:00,2019-04-25
2,2,2,9223372036854775807,1,652,1361762,208200314458056,33760627129,35281510359387,D42335A,...,34425237,23751202,229980251,1726277,10229119,30573789,414908351,2019-04-09,11:58:00,2019-04-25
3,3,3,9223372036854775807,486,171,1321509,208201402342131,33750343200,35356610164913,T21824A,...,36534765,15092588,810387875,657493,11811761,44763972,761837216,2019-04-10,07:36:00,2019-04-25
4,4,4,9223372036854775807,565,954,1089009,208201401415120,33699795932,35407009745539,D88865A,...,34222253,17539799,531237049,912788,7748843,40479310,564619822,2019-04-12,10:40:00,2019-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,149996,149996,7277825670196679680,451,214,81230,208202201200072,33650688697,35483109451938,D20434A,...,27955095,37561864,535806880,3517065,12784914,57686490,16751926,2019-04-29,06:02:00,2019-04-30
149997,149997,149997,7349883264234609664,483,187,97970,208201908153249,33663449963,35660508296467,D10223C,...,22165518,40643294,631628095,2352181,4415361,41089495,714638145,2019-04-29,10:41:00,2019-04-30
149998,149998,149998,9223372036854775807,283,810,98249,208201711161187,33621890103,35721209870907,T51102A,...,24516413,15029702,566933800,1260239,6585469,36437958,132415585,2019-04-29,10:46:00,2019-04-30
149999,149999,149999,9223372036854775807,696,327,97910,208202101098075,33619622058,86186204011457,L88342B,...,17122508,16494842,355066446,823109,7757267,31854366,816120043,2019-04-29,10:40:00,2019-04-30


In [2]:
cleaned_data_1['Total_Data_Usage'] = cleaned_data_1['Total UL (Bytes)'] + cleaned_data_1['Total DL (Bytes)']

# Define the criteria for the "Experienced Cluster"
experienced_cluster_criteria = (
    (cleaned_data_1['Total_Data_Usage'] > 500000000) &
    (cleaned_data_1['Dur. (ms)'] > 500) &
    ((cleaned_data_1['Total_Youtube'] > 10000000) | (cleaned_data_1['Total_Netflix'] > 10000000))
)

# Add the "Experienced Cluster" column based on the criteria
cleaned_data_1['Experienced_Cluster'] = experienced_cluster_criteria.astype(int)
cleaned_data_1['Experienced_Cluster']


0         0
1         1
2         0
3         1
4         1
         ..
149996    1
149997    1
149998    1
149999    0
150000    0
Name: Experienced_Cluster, Length: 150001, dtype: int64

In [3]:
# Calculate the total data usage
cleaned_data_1['Total_Data_Usage'] = cleaned_data_1['Total UL (Bytes)'] + cleaned_data_1['Total DL (Bytes)']

# Normalize the columns to a 0-1 range
cleaned_data_1['Total_Data_Usage_Norm'] = (cleaned_data_1['Total_Data_Usage'] - cleaned_data_1['Total_Data_Usage'].min()) / (cleaned_data_1['Total_Data_Usage'].max() - cleaned_data_1['Total_Data_Usage'].min())
cleaned_data_1['Dur_Norm'] = (cleaned_data_1['Dur. (ms)'] - cleaned_data_1['Dur. (ms)'].min()) / (cleaned_data_1['Dur. (ms)'].max() - cleaned_data_1['Dur. (ms)'].min())
cleaned_data_1['Total_Youtube_Norm'] = (cleaned_data_1['Total_Youtube'] - cleaned_data_1['Total_Youtube'].min()) / (cleaned_data_1['Total_Youtube'].max() - cleaned_data_1['Total_Youtube'].min())
cleaned_data_1['Total_Netflix_Norm'] = (cleaned_data_1['Total_Netflix'] - cleaned_data_1['Total_Netflix'].min()) / (cleaned_data_1['Total_Netflix'].max() -cleaned_data_1['Total_Netflix'].min())
cleaned_data_1['Total_Gaming_Norm'] = (cleaned_data_1['Total_Gaming'] - cleaned_data_1['Total_Gaming'].min()) / (cleaned_data_1['Total_Gaming'].max() - cleaned_data_1['Total_Gaming'].min())

# Define the weights
weights = {
    'Total_Data_Usage_Norm': 0.4,
    'Dur_Norm': 0.3,
    'Total_Youtube_Norm': 0.1,
    'Total_Netflix_Norm': 0.1,
    'Total_Gaming_Norm': 0.1
}

# Calculate the experienced score
cleaned_data_1['Experienced_Score'] = (
    weights['Total_Data_Usage_Norm'] * cleaned_data_1['Total_Data_Usage_Norm'] +
    weights['Dur_Norm'] * cleaned_data_1['Dur_Norm'] +
    weights['Total_Youtube_Norm'] * cleaned_data_1['Total_Youtube_Norm'] +
    weights['Total_Netflix_Norm'] * cleaned_data_1['Total_Netflix_Norm'] +
    weights['Total_Gaming_Norm'] * cleaned_data_1['Total_Gaming_Norm']
)



In [4]:
cleaned_data_1['Experienced_Score']

0         0.544759
1         0.749288
2         0.495001
3         0.792514
4         0.601117
            ...   
149996    0.479331
149997    0.519118
149998    0.426356
149999    0.290900
150000    0.367252
Name: Experienced_Score, Length: 150001, dtype: float64

In [5]:
cleaned_data_1

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Total_Email,Total_Other,Total_Data_Usage,Experienced_Cluster,Total_Data_Usage_Norm,Dur_Norm,Total_Youtube_Norm,Total_Netflix_Norm,Total_Gaming_Norm,Experienced_Score
0,9223372036854775807,2019-04-04 12:01:00,770,2019-04-25 14:35:00,662,1823652,208201448079117,33664962239,35521209507511,9.16456699548519E+015,...,40313283,180558843,345629377,0,0.341600,0.980734,0.405155,0.393722,0.340111,0.544759
1,9223372036854775807,2019-04-09 13:04:00,235,2019-04-25 08:15:00,606,1365104,208201909211140,33681854413,35794009006359,L77566A,...,54429437,541959383,707185356,1,0.731616,0.733164,0.870743,0.786415,0.709765,0.749288
2,9223372036854775807,2019-04-09 17:42:00,1,2019-04-25 11:58:00,652,1361762,208200314458056,33760627129,35281510359387,D42335A,...,30573789,414908351,307690973,0,0.300676,0.731360,0.761371,0.524455,0.267406,0.495001
3,9223372036854775807,2019-04-10 00:31:00,486,2019-04-25 07:36:00,171,1321509,208201402342131,33750343200,35356610164913,T21824A,...,44763972,761837216,889352748,1,0.928123,0.709627,0.808134,0.332467,0.943166,0.792514
4,9223372036854775807,2019-04-12 20:10:00,565,2019-04-25 10:40:00,954,1089009,208201401415120,33699795932,35407009745539,D88865A,...,40479310,564619822,607681403,1,0.624280,0.584100,0.756871,0.386729,0.618155,0.601117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,7277825670196679680,2019-04-29 07:28:00,451,2019-04-30 06:02:00,214,81230,208202201200072,33650688697,35483109451938,D20434A,...,57686490,16751926,631804110,1,0.650301,0.040000,0.617944,0.830680,0.623475,0.479331
149997,7349883264234609664,2019-04-29 07:28:00,483,2019-04-30 10:41:00,187,97970,208201908153249,33663449963,35660508296467,D10223C,...,41089495,714638145,705783925,1,0.730105,0.049038,0.489604,0.899005,0.735038,0.519118
149998,9223372036854775807,2019-04-29 07:28:00,283,2019-04-30 10:46:00,810,98249,208201711161187,33621890103,35721209870907,T51102A,...,36437958,132415585,627698629,1,0.645873,0.049189,0.541717,0.331072,0.659716,0.426356
149999,9223372036854775807,2019-04-29 07:28:00,696,2019-04-30 10:40:00,327,97910,208202101098075,33619622058,86186204011457,L88342B,...,31854366,816120043,401522016,0,0.401893,0.049006,0.377813,0.363559,0.413042,0.290900


In [6]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

cleaned_data_1['Start'] = pd.to_datetime(cleaned_data_1['Start'])
cleaned_data_1['End'] = pd.to_datetime(cleaned_data_1['End'])
# Calculate the total session duration in milliseconds
cleaned_data_1['Total_Session_Duration'] = (cleaned_data_1['End'] - cleaned_data_1['Start']).dt.total_seconds() * 1000
# Calculate the session frequency per user
session_counts = cleaned_data_1.groupby('MSISDN/Number').size().reset_index(name='Session_Frequency')
# Merge session frequency with the original data
cleaned_data_1 = cleaned_data_1.merge(session_counts, on='MSISDN/Number', how='left', suffixes=('', '_count'))
# Calculate total traffic (upload + download)
cleaned_data_1['Total_Traffic'] = cleaned_data_1['Total DL (Bytes)'] + cleaned_data_1['Total UL (Bytes)']
satisfaction_score = cleaned_data_1.groupby('IMSI').apply(lambda x: x['Total DL (Bytes)'].sum() / x['Total UL (Bytes)'].sum()).reset_index(name='Satisfaction_score')
cleaned_data_1 = cleaned_data_1.merge(satisfaction_score, on='IMSI', how='left')
customer_metrics = cleaned_data_1[['MSISDN/Number', 'Session_Frequency', 'Total_Session_Duration', 'Total_Traffic', 'Satisfaction_score']]
# Remove the Duplicates
customer_metrics = customer_metrics.drop_duplicates(subset='MSISDN/Number')
scaled_data = scaler.fit_transform(customer_metrics[['Session_Frequency', 'Total_Session_Duration', 'Total_Traffic', 'Satisfaction_score']])
kmeans = KMeans(n_clusters=3, random_state=42)
customer_metrics['Experience_cluster'] = kmeans.fit_predict(scaled_data)
customer_metrics['cluster'] = kmeans.labels_
print(customer_metrics.head())


   MSISDN/Number  Session_Frequency  Total_Session_Duration  Total_Traffic  \
0    33664962239                  2            1.823640e+09      345629377   
1    33681854413                  2            1.365060e+09      707185356   
2    33760627129                  1            1.361760e+09      307690973   
3    33750343200                  1            1.321500e+09      889352748   
4    33699795932                  1            1.089000e+09      607681403   

   Satisfaction_score  Experience_cluster  cluster  
0            8.022016                   1        1  
1           12.276447                   0        0  
2           10.034822                   1        1  
3           19.527843                   0        0  
4           14.766399                   0        0  


In [7]:
customer_metrics

,MSISDN/Number,Session_Frequency,Total_Session_Duration,Total_Traffic,Satisfaction_score,Experience_cluster,cluster
0,33664962239,2,1.823640e+09,345629377,8.022016,1,1
1,33681854413,2,1.365060e+09,707185356,12.276447,0,0
2,33760627129,1,1.361760e+09,307690973,10.034822,1,1
3,33750343200,1,1.321500e+09,889352748,19.527843,0,0
4,33699795932,1,1.089000e+09,607681403,14.766399,0,0
...,...,...,...,...,...,...,...
149994,33645655643,1,1.135800e+08,922173944,14.530583,0,0
149995,33668648496,1,5.958000e+07,872761860,15.238822,0,0
149996,33650688697,1,8.124000e+07,631804110,9.963330,0,0
149998,33621890103,1,9.828000e+07,627698629,16.979337,0,0


In [8]:
centroids = kmeans.cluster_centers_

# Identify the least engaged cluster (let's assume it has the lowest total traffic)
least_engaged_cluster = np.argmin(centroids[:, 2])

# Identify the worst experience cluster (let's assume it has the lowest satisfaction score)
worst_experience_cluster = np.argmin(centroids[:, 3])
print(worst_experience_cluster)
print(least_engaged_cluster)

2
1


In [9]:
# Assuming X is the feature matrix and y is the target variable (Satisfaction_Score)
X = customer_metrics[['Session_Frequency', 'Total_Session_Duration', 'Total_Traffic']]
y = customer_metrics['Satisfaction_score']

# Handle missing values by filling with the mean
X.fillna(X.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended for some models)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a linear regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Display the model performance
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)

Mean Squared Error: 34.12486541207376
R-squared: 0.4071626535160864
Coefficients: [-0.25962702  0.04600933  5.31169985]
Intercept: 11.971298762369278


In [10]:
# Aggregate average satisfaction score per engagement cluster
avg_satisfaction_engagement = customer_metrics.groupby('Experience_cluster')['Satisfaction_score'].mean()

# Aggregate average experience score per experience cluster
avg_experience_experience = cleaned_data_1.groupby('Experienced_Cluster')['Experienced_Score'].mean()

# Display the results
print("Average Satisfaction Score per Engagement Cluster:")
print(avg_satisfaction_engagement)

print("\nAverage Experience Score per Experience Cluster:")
print(avg_experience_experience)

Average Satisfaction Score per Engagement Cluster:
Experience_cluster
0    17.336717
1     6.886950
2     6.682742
Name: Satisfaction_score, dtype: float64

Average Experience Score per Experience Cluster:
Experienced_Cluster
0    0.252961
1    0.485418
Name: Experienced_Score, dtype: float64


# Save the tabel in local MYSQL

In [11]:
# import psycopg2
# from sqlalchemy import create_engine

# I have PostgreSQL installed locally and running
# username = "postgres"
# password = "Admin"
# host = "localhost"
# port = "5432"
# database_name = "telecom_db"

# Create a PostgreSQL connection string
# postgres_conn_str = f"postgresql://{username}:{password}@{host}:{port}/{database_name}"

# Create a SQLAlchemy engine
# engine = create_engine(postgres_conn_str)

# customer_metrics_table = customer_metrics[['MSISDN/Number', 'Engagement_Score']]
# df_table = df[['MSISDN/Number', 'Experience_Score']]

# Export the tables to PostgreSQL
# customer_metrics_table.to_sql('customer_metrics_table', engine, index=False, if_exists='replace')
# df_table.to_sql('df_table', engine, index=False, if_exists='replace')
# Run a select query
# query = "SELECT * FROM xdr_data"
# result = pd.read_sql_query(query, engine)

# Display the result (for verification)
# print(result)

# Check the database 

In [12]:

# Assuming 'Engagement_Score' is in customer_metrics and 'Experience_Score' is in df
# engagement_metrics = customer_metrics[['MSISDN/Number', 'Engagement_Score']]
# experience_metrics = df[['MSISDN/Number', 'Experience_Score']]

# Merge dataframes on 'MSISDN/Number'
# merged_df = pd.merge(engagement_metrics, experience_metrics, on='MSISDN/Number', how='inner')

# Fill missing values only for numerical columns
# merged_df[['Engagement_Score', 'Experience_Score']] = merged_df[['Engagement_Score', 'Experience_Score']].fillna(merged_df.mean())

# Create a PostgreSQL connection string
# username = "postgres"
# password = "Admin"
# host = "localhost"
# port = "5432"
# database_name = "telecom_db"

# Create a SQLAlchemy engine
# postgres_conn_str = f"postgresql://{username}:{password}@{host}:{port}/{database_name}"
# engine = create_engine(postgres_conn_str)

# final_table = merged_df[['MSISDN/Number', 'Engagement_Score', 'Experience_Score']]

# Export the final table to PostgreSQL
# final_table.to_sql('your_table_name', engine, index=False, if_exists='replace')

# Run a select query
# select_query = "SELECT * FROM your_table_name LIMIT 10"
# result = pd.read_sql(select_query, con=engine)

# Display the result
# print("Select Query Result:")
# print(result)

In [13]:
cleaned_data_1

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Total_Data_Usage_Norm,Dur_Norm,Total_Youtube_Norm,Total_Netflix_Norm,Total_Gaming_Norm,Experienced_Score,Total_Session_Duration,Session_Frequency,Total_Traffic,Satisfaction_score
0,9223372036854775807,2019-04-04 12:01:00,770,2019-04-25 14:35:00,662,1823652,208201448079117,33664962239,35521209507511,9.16456699548519E+015,...,0.341600,0.980734,0.405155,0.393722,0.340111,0.544759,1.823640e+09,2,345629377,8.022016
1,9223372036854775807,2019-04-09 13:04:00,235,2019-04-25 08:15:00,606,1365104,208201909211140,33681854413,35794009006359,L77566A,...,0.731616,0.733164,0.870743,0.786415,0.709765,0.749288,1.365060e+09,2,707185356,12.276447
2,9223372036854775807,2019-04-09 17:42:00,1,2019-04-25 11:58:00,652,1361762,208200314458056,33760627129,35281510359387,D42335A,...,0.300676,0.731360,0.761371,0.524455,0.267406,0.495001,1.361760e+09,1,307690973,10.034822
3,9223372036854775807,2019-04-10 00:31:00,486,2019-04-25 07:36:00,171,1321509,208201402342131,33750343200,35356610164913,T21824A,...,0.928123,0.709627,0.808134,0.332467,0.943166,0.792514,1.321500e+09,1,889352748,19.527843
4,9223372036854775807,2019-04-12 20:10:00,565,2019-04-25 10:40:00,954,1089009,208201401415120,33699795932,35407009745539,D88865A,...,0.624280,0.584100,0.756871,0.386729,0.618155,0.601117,1.089000e+09,1,607681403,14.766399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,7277825670196679680,2019-04-29 07:28:00,451,2019-04-30 06:02:00,214,81230,208202201200072,33650688697,35483109451938,D20434A,...,0.650301,0.040000,0.617944,0.830680,0.623475,0.479331,8.124000e+07,1,631804110,9.963330
149997,7349883264234609664,2019-04-29 07:28:00,483,2019-04-30 10:41:00,187,97970,208201908153249,33663449963,35660508296467,D10223C,...,0.730105,0.049038,0.489604,0.899005,0.735038,0.519118,9.798000e+07,2,705783925,18.117921
149998,9223372036854775807,2019-04-29 07:28:00,283,2019-04-30 10:46:00,810,98249,208201711161187,33621890103,35721209870907,T51102A,...,0.645873,0.049189,0.541717,0.331072,0.659716,0.426356,9.828000e+07,1,627698629,16.979337
149999,9223372036854775807,2019-04-29 07:28:00,696,2019-04-30 10:40:00,327,97910,208202101098075,33619622058,86186204011457,L88342B,...,0.401893,0.049006,0.377813,0.363559,0.413042,0.290900,9.792000e+07,1,401522016,12.552984


In [37]:
data_1 = cleaned_data_1['End'].astype(str) 
data_split = [entry.split() for entry in data_1]
split_df = pd.DataFrame(data_split, columns=["EndDate", "EndTime"])

# Assign the new columns to the original DataFrame
cleaned_data_1["EndDate"] = split_df["EndDate"]
cleaned_data_1["StartTime"] = split_df["EndTime"]
cleaned_data_1.drop(columns=["End"], inplace=True)
print(cleaned_data_1)
cleaned_data_1.to_csv('cleaned_dataset_1.csv',index=True)

        Unnamed: 0            Bearer Id  Start ms  End ms  Dur. (ms)  \
0                0  9223372036854775807       770     662    1823652   
1                1  9223372036854775807       235     606    1365104   
2                2  9223372036854775807         1     652    1361762   
3                3  9223372036854775807       486     171    1321509   
4                4  9223372036854775807       565     954    1089009   
...            ...                  ...       ...     ...        ...   
149996      149996  7277825670196679680       451     214      81230   
149997      149997  7349883264234609664       483     187      97970   
149998      149998  9223372036854775807       283     810      98249   
149999      149999  9223372036854775807       696     327      97910   
150000      150000  9223372036854775807       499     498     104608   

                   IMSI  MSISDN/Number            IMEI     Last Location Name  \
0       208201448079117    33664962239  35521209507511